# Aufgabenzuteilung

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/Job-Assignment.ipynb)

Eine Kreditratingagentur hat drei Mitarbeiter für ein Projekt zur Bewertung von strukturierten Finanzprodukten eingeteilt. Das Projekt hat drei wesentliche Unterprojekte: 

- Entwicklung von statistischen Modellen (Quantitativ)
- Recherche von Finanzprodukten, Literatur und Marktinformationen (Recherche)
- Entwicklung von qualitativen Modellen (Qualitativ)

Jeder Mitarbeiter muss ein Unterprojekt übernehmen. Die Mitarbeiter haben unterschiedliche Fähigkeiten und Präferenzen, die ihre Effizienz in den verschiedenen Unterprojekten beeinflussen. Die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten ist wie folgt (größerer Wert bedeutet höhere Effizienz):

In [ ]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

dt = pd.DataFrame({
    'Quantitativ': [53, 27, 13],
    'Recherche': [80, 47, 67],
    'Qualitativ': [53, 73, 47]
}, index=['Boyko', 'Sasho', 'Radi'])
dt

Note: you may need to restart the kernel to use updated packages.


,Quantitativ,Recherche,Qualitativ
Boyko,53,80,53
Sasho,27,47,73
Radi,13,67,47


Unsere Aufgabe ist es, die Mitarbeiter so zuzuweisen, dass die Gesamteffizienz maximiert wird.


## Das Modell

Jedes Modell besteht aus fünf Bestandteilen:

1. Mengen
2. Variablen
3. Parameter
4. Zielfunktion
5. Einschränkungen

In dieser Aufgabe haben wir zwei Mengen:

- Mitarbeiter ($M = \{\text{Boyko}, \text{Sasho}, \text{Radi}\}$)
- Unterprojekte ($P = \{\text{Quantitativ}, \text{Recherche}, \text{Qualitativ}\}$)

Die Zielvariablen sind:

$x_{ij} \in \{0, 1\}$: 1, wenn Mitarbeiter $i$ Unterprojekt $j$ übernimmt, 0 sonst (also binäre Variablen)

Die Parameter sind hier die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten:

$e_{ij}$: Effizienz von Mitarbeiter $i$ in Unterprojekt $j$

Die Zielfunktion ist die Gesamteffizienz:

$\max \sum_{i \in M} \sum_{j \in P} e_{ij} \cdot x_{ij}$

Das ist eine Kurzform für:

$$
\begin{align*}
\max & \quad e_{\text{Boyko}, \text{Quantitativ}} \cdot x_{\text{Boyko}, \text{Quantitativ}} + e_{\text{Boyko}, \text{Recherche}} \cdot x_{\text{Boyko}, \text{Recherche}} + e_{\text{Boyko}, \text{Qualitativ}} \cdot x_{\text{Boyko}, \text{Qualitativ}} \\
& + e_{\text{Sasho}, \text{Quantitativ}} \cdot x_{\text{Sasho}, \text{Quantitativ}} + e_{\text{Sasho}, \text{Recherche}} \cdot x_{\text{Sasho}, \text{Recherche}} + e_{\text{Sasho}, \text{Qualitativ}} \cdot x_{\text{Sasho}, \text{Qualitativ}} \\
& + e_{\text{Radi}, \text{Quantitativ}} \cdot x_{\text{Radi}, \text{Quantitativ}} + e_{\text{Radi}, \text{Recherche}} \cdot x_{\text{Radi}, \text{Recherche}} + e_{\text{Radi}, \text{Qualitativ}} \cdot x_{\text{Radi}, \text{Qualitativ}}
\end{align*}
$$

Die Einschränkungen sind:

1. Jeder Mitarbeiter muss genau ein Unterprojekt übernehmen:

$\sum_{j \in P} x_{ij} = 1 \quad \forall i \in M$

Dies ist eine Kurzform für:

$$
\begin{align*}
i = \text{Boyko} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Quantitativ}} + x_{\text{Boyko}, \text{Recherche}} + x_{\text{Boyko}, \text{Qualitativ}} = 1 \\
i = \text{Sasho} & \quad \Rightarrow \quad x_{\text{Sasho}, \text{Quantitativ}} + x_{\text{Sasho}, \text{Recherche}} + x_{\text{Sasho}, \text{Qualitativ}} = 1 \\
i = \text{Radi} & \quad \Rightarrow \quad x_{\text{Radi}, \text{Quantitativ}} + x_{\text{Radi}, \text{Recherche}} + x_{\text{Radi}, \text{Qualitativ}} = 1 \\
\end{align*}
$$

2. Jedes Unterprojekt muss von genau einem Mitarbeiter übernommen werden:

$\sum_{i \in M} x_{ij} = 1 \quad \forall j \in P$

Dies ist eine Kurzform für:

$$
\begin{align*}
j = \text{Quantitativ} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Quantitativ}} + x_{\text{Sasho}, \text{Quantitativ}} + x_{\text{Radi}, \text{Quantitativ}} = 1 \\
j = \text{Recherche} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Recherche}} + x_{\text{Sasho}, \text{Recherche}} + x_{\text{Radi}, \text{Recherche}} = 1 \\
j = \text{Qualitativ} & \quad \Rightarrow \quad x_{\text{Boyko}, \text{Qualitativ}} + x_{\text{Sasho}, \text{Qualitativ}} + x_{\text{Radi}, \text{Qualitativ}} = 1 \\
\end{align*}
$$

## Umsetzung


In [2]:

m = gp.Model('Aufgabenzuteilung')

# Die Variablen x_{ij} geben an, ob Aufgabe i an Person j zugewiesen wird

x = m.addVars(dt.index, dt.columns, vtype=GRB.BINARY, name='assign')

# Die Zielfunktion ist die Summe der Produkte der Qualifikationen und der Zuweisungen

m.setObjective(sum(dt.loc[i, j] * x[i, j] for i in dt.index for j in dt.columns), GRB.MAXIMIZE)

# Jeder Mitarbeiter kann nur eine Aufgabe erhalten

m.addConstrs((x.sum(i, '*') == 1 for i in dt.index),'Mitarbeiter')

# Jede Aufgabe kann nur einem Mitarbeiter zugewiesen werden

m.addConstrs((x.sum('*', j) == 1 for j in dt.columns), 'Aufgabe')

m.optimize()

# Die Lösung as pandas DataFrame

solution = pd.DataFrame(
    ((i, j, x[i, j].x) for i in dt.index for j in dt.columns),
    columns=['Mitarbeiter', 'Aufgabe', 'Zuweisung']
)
solution

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) i9-14900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xdc8c1267
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)
Found heuristic solution: objective 113.0000000
Found heuristic solution: objective 147.0000000
Found heuristic solution: objective 193.0000000

Root relaxation: cutoff, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

,Mitarbeiter,Aufgabe,Zuweisung
0,Boyko,Quantitative,1.0
1,Boyko,Recherche,-0.0
2,Boyko,Qualitative,-0.0
3,Sasho,Quantitative,-0.0
4,Sasho,Recherche,-0.0
5,Sasho,Qualitative,1.0
6,Radi,Quantitative,-0.0
7,Radi,Recherche,1.0
8,Radi,Qualitative,-0.0


In [7]:
# Um die Zuweisungen zu visualisieren, können wir die Tabelle umformen

solution.pivot(index='Mitarbeiter', columns='Aufgabe', values='Zuweisung')


Aufgabe,Qualitative,Quantitative,Recherche
Mitarbeiter,,,
Boyko,-0.0,1.0,-0.0
Radi,-0.0,-0.0,1.0
Sasho,1.0,-0.0,-0.0


In [3]:
m.write('Aufgabenzuteilung.lp')

with open('Aufgabenzuteilung.lp', 'r') as f:
    print(f.read())
    

\ Model Aufgabenzuteilung
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  53 assign[Boyko,Quantitative] + 80 assign[Boyko,Recherche]
   + 53 assign[Boyko,Qualitative] + 27 assign[Sasho,Quantitative]
   + 47 assign[Sasho,Recherche] + 73 assign[Sasho,Qualitative]
   + 13 assign[Radi,Quantitative] + 67 assign[Radi,Recherche]
   + 47 assign[Radi,Qualitative]
Subject To
 Mitarbeiter[Boyko]: assign[Boyko,Quantitative] + assign[Boyko,Recherche]
   + assign[Boyko,Qualitative] = 1
 Mitarbeiter[Sasho]: assign[Sasho,Quantitative] + assign[Sasho,Recherche]
   + assign[Sasho,Qualitative] = 1
 Mitarbeiter[Radi]: assign[Radi,Quantitative] + assign[Radi,Recherche]
   + assign[Radi,Qualitative] = 1
 Aufgabe[Quantitative]: assign[Boyko,Quantitative]
   + assign[Sasho,Quantitative] + assign[Radi,Quantitative] = 1
 Aufgabe[Recherche]: assign[Boyko,Recherche] + assign[Sasho,Recherche]
   + assign[Radi,Recherche] = 1
 Aufgabe[Qualitative]: assign[Boyko,Qualitative]


Einen alternativen Ansatz zur Modellierung wäre es, zuerst den Datensatz umzustrukturieren, so dass die Effizienz der Mitarbeiter in den verschiedenen Unterprojekten in einer Spalte steht (Langform). Dann könnten wir `x.prod()` verwenden und die Schleife in der Zielfunktion vermeiden.

In [ ]:
# Den Datensatz in Langform bringen. Das Ergebnis ist eine Serie mit einem MultiIndex

dt_long = dt.stack()
dt_long

Boyko  Quantitative    53
       Recherche       80
       Qualitative     53
Sasho  Quantitative    27
       Recherche       47
       Qualitative     73
Radi   Quantitative    13
       Recherche       67
       Qualitative     47
dtype: int64

In [25]:
dt_long.index

MultiIndex([('Boyko', 'Quantitative'),
            ('Boyko',    'Recherche'),
            ('Boyko',  'Qualitative'),
            ('Sasho', 'Quantitative'),
            ('Sasho',    'Recherche'),
            ('Sasho',  'Qualitative'),
            ( 'Radi', 'Quantitative'),
            ( 'Radi',    'Recherche'),
            ( 'Radi',  'Qualitative')],
           )

In [28]:
# Auf die einzelnen Werte des MultiIndex zugreifen

dt_long.index.levels[0]

Index(['Boyko', 'Sasho', 'Radi'], dtype='object')

In [29]:
dt_long.index.levels[1]

Index(['Quantitative', 'Recherche', 'Qualitative'], dtype='object')

In [32]:
# Die Serie in ein Dictionary umwandeln

dt_long.to_dict()

{('Boyko', 'Quantitative'): 53,
 ('Boyko', 'Recherche'): 80,
 ('Boyko', 'Qualitative'): 53,
 ('Sasho', 'Quantitative'): 27,
 ('Sasho', 'Recherche'): 47,
 ('Sasho', 'Qualitative'): 73,
 ('Radi', 'Quantitative'): 13,
 ('Radi', 'Recherche'): 67,
 ('Radi', 'Qualitative'): 47}

In [34]:
list((x.sum(i, '*') == 1 for i in dt_long.index.levels[0]))

[<gurobi.TempConstr: assign[Boyko,Quantitative] + assign[Boyko,Recherche] + assign[Boyko,Qualitative] == 1>,
 <gurobi.TempConstr: assign[Sasho,Quantitative] + assign[Sasho,Recherche] + assign[Sasho,Qualitative] == 1>,
 <gurobi.TempConstr: assign[Radi,Quantitative] + assign[Radi,Recherche] + assign[Radi,Qualitative] == 1>]

In [30]:
m1 = gp.Model('Aufgabenzuteilung 1')
m1.Params.LogToConsole = 0

x = m1.addVars(dt_long.index, vtype=GRB.BINARY, name='assign')

m1.setObjective(x.prod(dt_long.to_dict()), GRB.MAXIMIZE)

m1.addConstrs((x.sum(i, '*') == 1 for i in dt_long.index.levels[0]), 'Mitarbeiter')
m1.addConstrs((x.sum('*', j) == 1 for j in dt_long.index.levels[1]), 'Unterprojekt')

m1.write('Aufgabenzuteilung 1.lp')

with open('Aufgabenzuteilung 1.lp', 'r') as f:
    print(f.read())

m1.optimize()

solution1 = pd.DataFrame(
    ((i, j, x[i, j].x) for i, j in dt_long.index),
    columns=['Mitarbeiter', 'Unterprojekt', 'Zuweisung']
)
solution1.pivot(index='Mitarbeiter', columns='Unterprojekt', values='Zuweisung')

Set parameter LogToConsole to value 0
\ Model Aufgabenzuteilung 1
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  53 assign[Boyko,Quantitative] + 80 assign[Boyko,Recherche]
   + 53 assign[Boyko,Qualitative] + 27 assign[Sasho,Quantitative]
   + 47 assign[Sasho,Recherche] + 73 assign[Sasho,Qualitative]
   + 13 assign[Radi,Quantitative] + 67 assign[Radi,Recherche]
   + 47 assign[Radi,Qualitative]
Subject To
 Mitarbeiter[Boyko]: assign[Boyko,Quantitative] + assign[Boyko,Recherche]
   + assign[Boyko,Qualitative] = 1
 Mitarbeiter[Sasho]: assign[Sasho,Quantitative] + assign[Sasho,Recherche]
   + assign[Sasho,Qualitative] = 1
 Mitarbeiter[Radi]: assign[Radi,Quantitative] + assign[Radi,Recherche]
   + assign[Radi,Qualitative] = 1
 Unterprojekt[Quantitative]: assign[Boyko,Quantitative]
   + assign[Sasho,Quantitative] + assign[Radi,Quantitative] = 1
 Unterprojekt[Recherche]: assign[Boyko,Recherche] + assign[Sasho,Recherche]
   + assign[Radi,Recherche] = 1

Unterprojekt,Qualitative,Quantitative,Recherche
Mitarbeiter,,,
Boyko,-0.0,1.0,-0.0
Radi,-0.0,-0.0,1.0
Sasho,1.0,-0.0,-0.0
